In [1]:
import io
import os.path
import pandas as pd
import pickle

df = pd.read_csv('data/data_big.csv')

In [2]:
df = df.dropna() # remove nans
df.head()

,id,name,description,categories,genres
0,795720,"Warhammer 40,000: Space Wolf - Fall of Kanak",When the frontline squads of the Word Bearers ...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ..."
1,795731,War Thunder - Plagis' Spitfire LF Mk. IX,"This Pack includes:<br><br><ul class=""bb_ul""><...","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ..."
2,795732,War Thunder - T29 Pack,"This Pack includes:<br><br><ul class=""bb_ul""><...","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ..."
3,795910,Montero,General description:<br>Montero is a fantasy m...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i..."
4,795920,SALVATOR,Story:<br />\r\n<br />\r\nAll souls around of ...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}, {'id': ..."


In [3]:
#create list of sentences
docs = []
for line in df['description']:
    docs.append(line)

In [4]:
print(len(docs))
print(docs[0][:500]) #look at the first document

41125
When the frontline squads of the Word Bearers legion landed on Kanak and proceeded to capture the planet’s key installations, they encountered fierce resistance in the form of Kanak Skull Takers loyal to the Imperium. But alas, in spite of the defenders’ inhuman heroism and tenacity, their resistance amounted to nothing but a brief hindrance for the advance of the forces of Chaos.<br> <br> <br> This DLC allows you to witness the events of those long gone days from an unusual perspective.<br> <br


In [5]:
import preprocess as pre
docs_clean = [pre.preprocess(doc) for doc in docs]

In [6]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs_clean, min_count=20)
for idx in range(len(docs_clean)):
    for token in bigram[docs_clean[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs_clean[idx].append(token)

D:\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs_clean)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=30, no_above=0.7)

In [8]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs_clean]

In [9]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 10667
Number of documents: 41125


In [10]:
if(not os.path.isfile('model.p')):
    %%time
    # Train LDA model.
    from gensim.models import LdaModel

    # Set training parameters.
    num_topics = 25 #number of topics, similar to K in k-means
    chunksize = 2000 # Number of documents to be used in each training chunk.
    passes = 30 #Number of passes through the corpus during training.
    iterations = 400 #Maximum number of iterations through the corpus when inferring the topic distribution of a corpus.
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    # Make a index to word dictionary.
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token

    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )
else:
    model = pickle.load(open('model.p','rb'),encoding='uft-8')
    num_topics = 25

In [15]:
%%time
#import pyLDAvis.sklearn Sklearn version
import pyLDAvis
import pyLDAvis.gensim_models
 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)
vis

D:\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


Wall time: 17 s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.111097  0.012445       1        1   3.277318
1      0.149671 -0.119756       2        1   5.082316
2      0.069078 -0.012813       3        1   9.654704
3     -0.043985 -0.055232       4        1   2.184939
4     -0.193664 -0.062382       5        1   0.962055
5      0.133112  0.092312       6        1  13.826183
6      0.062575  0.034178       7        1   1.643893
7     -0.059419  0.035500       8        1   1.994452
8     -0.196556 -0.090865       9        1   1.132220
9     -0.091734  0.151554      10        1   2.934297
10     0.138045  0.061835      11        1   6.595039
11    -0.061634 -0.217756      12        1   1.696863
12     0.131903 -0.063468      13        1   7.748174
13    -0.013020  0.090505      14        1   2.200665
14    -0.091225 -0.128295      15        1   0.820846
15     0.039690  0.140985      16        1   7.444564
16    -0.098223 -0.163441      17        1   2.687033
17    -0.185526  0.234572      18        1   2.304360
18     0.141233  0.012581      19        1   3.304585
19     0.064967 -0.191156      20        1   3.026682
20     0.053632  0.131402      21        1   2.462095
21     0.192130 -0.029402      22        1   5.300078
22    -0.220584 -0.027731      23        1   1.192112
23     0.150302  0.052477      24        1   9.027800
24    -0.181865  0.111953      25        1   1.496727, topic_info=           Term          Freq         Total Category  logprob  loglift
268       level  19174.000000  19174.000000  Default  30.0000  30.0000
406      puzzle  11297.000000  11297.000000  Default  29.0000  29.0000
288      player  21635.000000  21635.000000  Default  28.0000  28.0000
23        enemy  15558.000000  15558.000000  Default  27.0000  27.0000
191      weapon  10360.000000  10360.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
11    character   1309.980358  15868.075246  Topic25  -3.7666   1.7076
892     product    448.738981   1635.866459  Topic25  -4.8379   2.9084
2008   required    408.928485   1357.835536  Topic25  -4.9308   3.0018
473      making    431.224173   2384.842414  Topic25  -4.8777   2.4916
1636      avoid    386.030210   2015.075326  Topic25  -4.9884   2.5494

[1557 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
9114      5  0.992840    18th
545       2  0.071844      2d
545       3  0.094230      2d
545      16  0.109848      2d
545      19  0.522168      2d
...     ...       ...     ...
1252      4  0.337189   young
1252      5  0.033279   young
1252     13  0.018837   young
1252     20  0.050233   young
2104      2  0.999377  zombie

[3596 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])

In [12]:
for index, score in sorted(model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, model.print_topic(index, 10)))


Score: 0.18523678183555603	 
Topic: 0.015*"build" + 0.010*"tank" + 0.010*"mission" + 0.010*"building" + 0.010*"city" + 0.009*"resource" + 0.009*"war" + 0.007*"unit" + 0.007*"map" + 0.007*"system"

Score: 0.18419644236564636	 
Topic: 0.032*"story" + 0.015*"adventure" + 0.011*"life" + 0.010*"year" + 0.009*"find" + 0.008*"one" + 0.007*"character" + 0.006*"two" + 0.006*"hidden" + 0.005*"past"

Score: 0.15340666472911835	 
Topic: 0.074*"player" + 0.034*"mode" + 0.033*"play" + 0.019*"team" + 0.019*"friend" + 0.012*"multiplayer" + 0.011*"online" + 0.010*"single" + 0.010*"opponent" + 0.008*"match"

Score: 0.14459705352783203	 
Topic: 0.020*"land" + 0.018*"kingdom" + 0.017*"king" + 0.017*"power" + 0.016*"god" + 0.015*"ancient" + 0.013*"castle" + 0.011*"warrior" + 0.011*"knight" + 0.011*"evil"

Score: 0.06901519000530243	 
Topic: 0.065*"enemy" + 0.051*"weapon" + 0.020*"attack" + 0.018*"fight" + 0.015*"gun" + 0.012*"action" + 0.011*"kill" + 0.011*"shooter" + 0.011*"damage" + 0.011*"zombie"

Scor

In [45]:
best_tags = {1:'Action', 2:'Adventure', 3:'Anime',  6:'Puzzle',  7:'Simulation', 8: 'Great Soundtrack', 9: 'RPG', 11: 'Space', 12: 'Casual', 13: ['Fantasy', 'RPG'],
             14: 'Card Game', 15: 'Simulation', 19: ['Fantasy','RPG'], 21: 'Adventure', 23: 'Strategy'}

In [43]:
my_list=[]
t = best_tags.get(13)
for x in t:
    my_list.append((x,0.1))
print(my_list)

[('Fantasy', 0.1), ('RPG', 0.1)]


In [38]:
top_topics = model.get_document_topics(corpus[0],minimum_probability=0.1)
tags = []
for topic in top_topics:
    t = best_tags.get(topic[0])
    if(t != None):
        if(not type(t) == list):
            tags.append((t, topic[1]))
        else:
            for x in t:
                tags.append((x, topic[1]))
print(sorted(tags, key = lambda tup : tup[1], reverse=True))

[('Strategy', 0.18523774), ('Adventure', 0.18418725), ('Fantasy', 0.14459723)]


In [29]:
df['genres'][0]

"[{'id': '1', 'description': 'Action'}, {'id': '2', 'description': 'Strategy'}]"

In [30]:
import ast
df['genres_ex']=df['genres'].apply(lambda x: [dict['description'] for dict in ast.literal_eval(x)])

In [49]:
score = 0
j = 0
for i, row in df.iterrows():
    top_topics = model.get_document_topics(corpus[j], minimum_probability=0.1)
    tags = []
    for topic in top_topics:
        t = best_tags.get(topic[0])
        if(t != None):
            if(not type(t) == list):
                tags.append((t, topic[1]))
            else:
                for x in t:
                    tags.append((x, topic[1]))
    if(len(tags)!=0):
        for tag in tags:
            if(tag[0] in row['genres_ex']):
                score+=1
                break
    j+=1

print(score/len(df))

[('RPG', 0.13397677), ('Strategy', 0.17404933)]
['Action', 'Free to Play', 'Massively Multiplayer', 'Simulation']
[('RPG', 0.13281636), ('Strategy', 0.21427818)]
['Action', 'Free to Play', 'Massively Multiplayer', 'Simulation']
[('Adventure', 0.19060372), ('Space', 0.10787444), ('Strategy', 0.21764795)]
['Indie', 'Massively Multiplayer', 'RPG', 'Simulation']
[('Puzzle', 0.10827335), ('Simulation', 0.17782842)]
['Action', 'Casual', 'Indie']
[('Adventure', 0.13677922)]
['Action', 'Indie']
[('Simulation', 0.17870022)]
['Tutorial']
[('Simulation', 0.1786709)]
['Tutorial']
[('Simulation', 0.18340048)]
['Tutorial']
[('Simulation', 0.18340033)]
['Tutorial']
[('Adventure', 0.14100546), ('Strategy', 0.112872116)]
['Action']
[('Adventure', 0.17492892)]
['Action', 'Indie']
[('Casual', 0.13026792)]
['Action', 'Adventure', 'Indie']
[('Adventure', 0.21174946), ('Anime', 0.14296024)]
['Sexual Content', 'Indie', 'Simulation']
[('Anime', 0.10203514), ('Strategy', 0.16333435)]
['Action', 'Indie', 'Simul